In [1]:
!pip install -U langchain_community tiktoken langchain-groq langchainhub chromadb langchain langgraph langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.4 MB/s eta 0:00:0

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [37]:
from google.colab import userdata

In [38]:
groq_api_key=userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"]=groq_api_key

In [39]:
TAVILY_API_KEY=userdata.get('TAVILY_API_KEY')

In [42]:
os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY

In [52]:
import string
import chromadb
import numpy as np
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.tools import TavilySearchResults  # ✅ Tavily for Web Search

# 🔹 Initialize VectorDB with HF embeddings
chroma_client = chromadb.PersistentClient(path="./chroma_db")
embedding_model = embeddings  # Embedding model for vectorization
vectorstore = Chroma(
    collection_name="rag-chatbot",
    embedding_function=embedding_model,
    client=chroma_client,
)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # Retrieve top 3 similar results

# 🔹 Initialize LLM
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")

# 🔹 Setup Tavily API (Get API Key from https://tavily.com/)
tavily = TavilySearchResults(api_key=TAVILY_API_KEY)

# 🔹 Normalize query text for better matching
def normalize_query(query: str) -> str:
    query = query.strip().lower()
    query = query.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    return query

# 🔹 Compute cosine similarity between vectors
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 🔹 Web Search for Research Queries
def fetch_web_research(query):
    print("🌍 Research query detected! Performing web search...")
    search_results = tavily.run(query)  # Perform web search
    print("🔍 Tavily Response:", search_results)  # Debugging step

    if isinstance(search_results, dict) and "results" in search_results:
        results_list = search_results["results"]  # Extract actual list
    elif isinstance(search_results, list):
        results_list = search_results  # Already in correct format
    else:
        print("⚠️ Unexpected Tavily response format!")
        return "No relevant research found."

    # Extract top 3 snippets safely
    research_summary = "\n".join([result["content"] for result in search_results[:3] if "content" in result])


    return research_summary if research_summary else "No relevant research found."

# 🔹 Fetch from VectorDB or generate new response based on similarity threshold
def get_response(query, retriever, vectorstore, similarity_threshold=0.8):
    query = normalize_query(query)  # Normalize query

    # Step 1: If query contains "research", perform web search
    if "research" in query:
        research_response = fetch_web_research(query)
        new_doc = Document(page_content=research_response, metadata={"query": query})
        vectorstore.add_documents([new_doc])  # Store or replace in VectorDB
        return research_response

    # Step 2: Check if a similar response exists in VectorDB
    results = retriever.invoke(query)

    if results:
        query_embedding = embedding_model.embed_query(query)  # Compute embedding of new query

        for res in results:
            stored_query = normalize_query(res.metadata["query"])
            stored_embedding = embedding_model.embed_query(stored_query)  # Compute embedding of stored query

            similarity_score = cosine_similarity(query_embedding, stored_embedding)
            print(f"🔍 Similarity with '{stored_query}': {similarity_score:.2f}")

            # If the stored query is semantically similar, return its response
            if similarity_score >= similarity_threshold:
                print("✅ Answer found in VectorDB!")
                return res.page_content

    # Step 3: If no similar response is found, query LLM
    print("🤖 Querying LLM...")
    llm_response = llm.invoke(query).content

    # Step 4: Store new question-answer pair in VectorDB
    new_doc = Document(page_content=llm_response, metadata={"query": query})
    vectorstore.add_documents([new_doc])
    print("📝 Response stored in VectorDB.")

    return llm_response

# 🔹 Continuous Chatbot Loop
def chatbot():
    print("💬 Chatbot is running! Type 'exit' to quit.")
    while True:
        user_input = input("\nUser: ")
        if user_input.lower() in ["exit", "quit", "q"]:
            print("👋 Goodbye!")
            break

        response = get_response(user_input, retriever, vectorstore)
        print(f"🤖 Assistant: {response}")

# Run chatbot
chatbot()


💬 Chatbot is running! Type 'exit' to quit.

User: research about deep learning
🌍 Research query detected! Performing web search...
🔍 Tavily Response: [{'title': 'Deep learning: systematic review, models, challenges, and research ...', 'url': 'https://link.springer.com/article/10.1007/s00521-023-08957-4', 'content': 'Deep learning: systematic review, models, challenges, and research directions | Neural Computing and Applications In this survey, the deep transfer learning techniques are classified based on the generalization viewpoints between deep learning models and domains into four categories, namely, instance, feature representation, model parameter, and relational knowledge-based techniques. Addressing these challenges requires a multidisciplinary approach involving data collection and preprocessing techniques, algorithmic enhancements, fairness-aware model training, interpretability methods, safe learning, robust models to adversarial attacks, and collaboration with domain experts